In [1]:
import os 
import pandas as pd 

## Load Data

Note - need to run analysis_video_vs_mat_and_outcomes script on both home and in-person videos first. That script created merged file used below 

In [2]:
version = '001'

In [3]:
output_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                           version, 
                          'video_visit_participant_counts')

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [4]:
bw_df = pd.read_excel(r'C:\Users\mmccu\AppData\Local\Temp\ccsecure\2024_10_15_BrainWalk_AllData_Long_MM.xlsx', 
                     index_col = None, 
                     usecols = ['bw_id', 'record_id', 'visit_date', 'demoEHR_DiseaseDuration',
                                'demoEHR_GENDER', 'demoEHR_Age', 'demographic_diagnosis', 'bingoEHR_DX_MS DX', 'demoEHR_REC_1'])

In [5]:
# zeno videos 
# all in-person videos 
zv_all_videos_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis\all_zeno_videos.csv'
zv_all_videos_df = pd.read_csv(zv_all_videos_path, index_col = 0)
zv_all_videos_df.rename(columns={'bw_id': 'id_video'}, inplace=True)


# included video .csv files - all videos included in analysis 
zv_included_videos_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code',
                                       'gait_bw_zeno_outputs_' + version,
                                       'gait_bw_zeno_outputs_' + version + '_included_videos.csv') 
zv_included_videos_df = pd.read_csv(zv_included_videos_path, index_col = 0)

# data merged w brainwalk 
# fast walk
zv_merged_fw_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis', 
                                 version, 
                                 'zv_bw_merged_gait_vertical_FW_1.csv')
zv_merged_fw_df = pd.read_csv(zv_merged_fw_path, index_col = 0)

# preferred walk 
zv_merged_pws_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis', 
                                 version, 
                                 'zv_bw_merged_gait_vertical_PWS_1.csv')
zv_merged_pws_df = pd.read_csv(zv_merged_pws_path, index_col = 0)

In [6]:
# Home videos 
hv_all_videos_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis\all_home_videos.csv'
hv_all_videos_df = pd.read_csv(hv_all_videos_path, index_col = 0)
hv_all_videos_df.rename(columns={'bw_id': 'id_video'}, inplace=True)

# included video .csv files - all videos included in analysis 
hv_included_videos_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code',
                                       'gait_bw_home_outputs_' + version,
                                       'gait_bw_home_outputs_' + version + '_included_videos.csv') 
hv_included_videos_df = pd.read_csv(hv_included_videos_path, index_col = 0)

# data merged w brainwalk  
hv_merged_bw_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis', 
                                 version, 
                                 'hv_bw_merged.csv')
hv_merged_bw_path_df = pd.read_csv(hv_merged_bw_path, index_col = 0)

## Merging Functions 
Merge video dates and ids with brainwalk data --> MS vs HC counts 

In [7]:
# task = gait_vertical_PWS_1 or gait_vertical_FW_1
def merge_bw_zv(bw_df, zv_df):
    
    # filter bw ids dataset to only those included in video data set 
    zv_in_bw_df = bw_df[bw_df['bw_id'].isin(zv_df['id_video'])]

    print('total bw rows with id in video dataset') 
    print(len(zv_in_bw_df))

    # merge bw data set rows with zeno videos rows 
        # merge bw data set rows with zeno videos rows 
        # id and date needs to be the same 
        # should only use each brainwalk visit once - once PWS_1 video per person 

    merged_bw_zv = []

    # Loop through each row in zv_df
    for index, zv_row in zv_df.iterrows():
        current_id = zv_row['id_video']
        current_date = zv_row['video_date']
        zv_row_df = pd.DataFrame([zv_row])

        # Find rows in brainwalk data set with same id and same date as current zv data 
        zv_in_bw_current_id_rows = zv_in_bw_df[(zv_in_bw_df['bw_id'] == current_id) & (zv_in_bw_df['visit_date'] == current_date)]
        #zv_in_bw_current_id_date_rows = zv_in_bw_current_id_rows[zv_in_bw_current_id_rows['visit_date'] == current_date]
   
        if len(zv_in_bw_current_id_rows) == 1: 
            bw_row_to_merge = zv_in_bw_current_id_rows
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)
        
        # if more than one row for the id and date, pick one with least na values 
        elif len(zv_in_bw_current_id_rows) > 1:
            bw_row_to_merge = zv_in_bw_current_id_rows.loc[[zv_in_bw_current_id_rows.isna().sum(axis=1).idxmin()]]
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)
            print('multiple rows in bw dataset for the id and date combo')
            print(current_id)
            print(current_date)

        else: 
            print('No matching id and daterow from video vs mat')
            print(current_id)
            print(current_date)


    # merge all bw and zv data together 
    merged_bw_zv_df = pd.concat(merged_bw_zv)
    merged_bw_zv_df = merged_bw_zv_df.reset_index(drop=True) # reset index 

    # check same ID for each row 
    print('mismatched zeno video vs brainwalk id')
    print(sum(merged_bw_zv_df['id_video'] != merged_bw_zv_df['bw_id']))

    print('mismatched zeno video vs brainwalk date')
    print(sum(merged_bw_zv_df['video_date'] != merged_bw_zv_df['visit_date']))

    return merged_bw_zv_df

## Summarize Video Counts

### All Videos ran through pose estimation 

In [8]:
#  total number of videos into algorithm and total # videos included 
    # for figure for each iteration of linear walking algorithm 

# Zeno Videos 
zv_total_videos = zv_all_videos_df['file_name'].count()
# preferred walking speed 
zv_total_pws_videos_df = zv_all_videos_df.loc[zv_all_videos_df['file_name'] == 'gait_vertical_PWS_1']
zv_total_pws_videos = zv_total_pws_videos_df['file_name'].count()
# fast walkign speed 
zv_total_fw_videos_df = zv_all_videos_df.loc[zv_all_videos_df['file_name'] == 'gait_vertical_FW_1']
zv_total_fw_videos = zv_total_fw_videos_df['file_name'].count()

# Home Videos -----------------------------------------------------
# There originally were XX right turning and XX left turning videos from XX participants 
hv_total_videos = hv_all_videos_df['file_name'].count()
#right 
hv_total_r_videos_df = hv_all_videos_df.loc[hv_all_videos_df['file_name'].str.contains('gait_vertical_right')]
hv_total_r_videos = hv_total_r_videos_df['file_name'].count()
# left 
hv_total_l_videos_df = hv_all_videos_df.loc[hv_all_videos_df['file_name'].str.contains('gait_vertical_left')]
hv_total_l_videos = hv_total_l_videos_df['file_name'].count()

In [9]:
# for all Zeno videos 
# A total of XX adults with a diagnosis of Multiple Sclerosis and a total 
        # of XX healthy controls were recruited from the University of California, San Francisco (UCSF) 

zv_all_videos_df['video_date'] = pd.to_datetime(zv_all_videos_df['video_date'].str.replace('_', '-'), format='%Y-%m-%d')

# merge bw data with all zeno video data 
zv_bw_all_videos_df = merge_bw_zv(bw_df, zv_all_videos_df)
zv_bw_all_videos_df.to_csv(os.path.join(output_path, 'all_zv_videos_merged_w_bw.csv'))

# for all Home videos - as of Jan 13/2025 - only collected home videos from participants with MS 

total bw rows with id in video dataset
331
No matching id and daterow from video vs mat
BW-0010
2022-10-05 00:00:00
No matching id and daterow from video vs mat
BW-0010
2022-10-05 00:00:00
multiple rows in bw dataset for the id and date combo
BW-0010
2023-10-18 00:00:00
multiple rows in bw dataset for the id and date combo
BW-0010
2023-10-18 00:00:00
No matching id and daterow from video vs mat
BW-0022
2023-04-18 00:00:00
No matching id and daterow from video vs mat
BW-0022
2023-04-18 00:00:00
No matching id and daterow from video vs mat
BW-0031
2023-04-21 00:00:00
No matching id and daterow from video vs mat
BW-0031
2023-04-21 00:00:00
No matching id and daterow from video vs mat
BW-0036
2024-04-02 00:00:00
No matching id and daterow from video vs mat
BW-0036
2024-04-02 00:00:00
No matching id and daterow from video vs mat
BW-0044
2023-05-02 00:00:00
No matching id and daterow from video vs mat
BW-0044
2023-05-02 00:00:00
multiple rows in bw dataset for the id and date combo
BW-0044
2

In [10]:
# count number of videos with bw data - should equal total videos 

# Zeno Videos 
zv_bw_total_videos = zv_bw_all_videos_df['file_name'].count()
vids_missing_w_bw_merge = zv_total_videos - zv_bw_total_videos 

# Home Videos - all MS, didn't need to merge 

In [11]:
# Zeno Videos -  count unique ID w MS  (n participants)
zv_bw_all_videos_MS_df = zv_bw_all_videos_df.loc[zv_bw_all_videos_df['demographic_diagnosis'] == 'MS']
zv_all_videos_MS_participant_count = zv_bw_all_videos_MS_df['id_video'].nunique()

# Zeno Videos - count unique ID w HC  (n participants)
zv_bw_all_videos_HC_df = zv_bw_all_videos_df.loc[zv_bw_all_videos_df['demographic_diagnosis'] == 'HC']
zv_all_videos_HC_participant_count = zv_bw_all_videos_HC_df['id_video'].nunique()

# home videos - all MS, (n participants)
hv_all_videos_participant_count = hv_all_videos_df['id_video'].nunique()
hv_all_videos_participant_count

35

In [12]:
# There originally were XX PWS and XX FW videos from XX participants with MS 
# and XX PW videos and XX FW videos from XX healthy controls

# Zeno videos - FW vs PWS in pwMS 
zv_all_videos_MS_pws_df = zv_bw_all_videos_MS_df.loc[zv_bw_all_videos_MS_df['file_name'] == 'gait_vertical_PWS_1']
zv_all_videos_MS_pws_count = zv_all_videos_MS_pws_df['file_name'].count() 

zv_all_videos_MS_fw_df = zv_bw_all_videos_MS_df.loc[zv_bw_all_videos_MS_df['file_name'] == 'gait_vertical_FW_1']
zv_all_videos_MS_fw_count = zv_all_videos_MS_fw_df['file_name'].count()  

# Zeno videos - FW vs PWS in HC 
zv_all_videos_HC_pws_df = zv_bw_all_videos_HC_df.loc[zv_bw_all_videos_HC_df['file_name'] == 'gait_vertical_PWS_1']
zv_all_videos_HC_pws_count = zv_all_videos_HC_pws_df['file_name'].count() 

zv_all_videos_HC_fw_df = zv_bw_all_videos_HC_df.loc[zv_bw_all_videos_HC_df['file_name'] == 'gait_vertical_FW_1']
zv_all_videos_HC_fw_count = zv_all_videos_HC_fw_df['file_name'].count() 

In [13]:
# from dates XX to XX.

# Zeno 
zv_all_videos_first_date = zv_bw_all_videos_df['video_date'].min()
zv_all_videos_last_date = zv_bw_all_videos_df['video_date'].max()

# Home 
hv_all_videos_first_date = hv_all_videos_df['video_date'].min()
hv_all_videos_last_date = hv_all_videos_df['video_date'].max()

In [14]:
#At the time of analysis, XX PwMS and XX healthy controls completed a single visit 
# and XX PwMS and XX healthy controls greater than one visit (baseline and yearly follow up)

# from all videos file:  unique date and id combo --> check for duplicate bw_id 

# Zeno Videos --------------------------------------------
# number of visits (unique id and date combo)
zv_all_vids_unique_date_id_df = zv_bw_all_videos_df.drop_duplicates(subset = ['video_date', 'id_video'])
zv_all_unique_visits = zv_all_vids_unique_date_id_df['file_name'].count()

# participants with MS 
zv_all_vids_unique_visit_ms = zv_all_vids_unique_date_id_df.loc[zv_all_vids_unique_date_id_df['demographic_diagnosis'] == 'MS']

# number of IDs repeated = multiple visits 
zv_multiple_visits_ms_count = zv_all_vids_unique_visit_ms['id_video'].value_counts()[lambda x: x > 1].count()
zv_single_visits_ms_count = zv_all_vids_unique_visit_ms['id_video'].value_counts()[lambda x: x == 1].count()

# HC 
zv_all_vids_unique_visit_hc = zv_all_vids_unique_date_id_df.loc[zv_all_vids_unique_date_id_df['demographic_diagnosis'] == 'HC']

# number of IDs repeated = multiple visits 
zv_multiple_visits_hc_count = zv_all_vids_unique_visit_hc['id_video'].value_counts()[lambda x: x > 1].count()
zv_single_visits_hc_count = zv_all_vids_unique_visit_hc['id_video'].value_counts()[lambda x: x == 1].count()

# Home Videos ----------------------------------------------
# number of visits (unique id and date combo - set of home videos sent back on one day)
hv_all_vids_unique_date_id_df = hv_all_videos_df.drop_duplicates(subset = ['video_date', 'id_video'])
hv_all_unique_visits = hv_all_vids_unique_date_id_df['file_name'].count()

# number of IDs repeated = multiple visits 
hv_multiple_visits_count = hv_all_vids_unique_date_id_df['id_video'].value_counts()[lambda x: x > 1].count()
hv_single_visits_count = hv_all_vids_unique_date_id_df['id_video'].value_counts()[lambda x: x == 1].count()

### videos included in analysis 
- included = videos included after linear walking algorithm
- analyzed = included videos with matched BW visit date and data --> need to QA BW dataset 

In [15]:
## Check - do .._bw_merged included videos match the included and excluded file numbers? any lost in the merge with BW? 

# count num included videos from included .csv then substract # videos in merged .csv (save # lost in merge, same as total values) 

# Zeno ----------------------------------------- 
# total included videos - from original script, true # of included vids before merging with brainwalk 
zv_included_vids_count = zv_included_videos_df['included_vids'].count()

# total included videos: pws 
zv_included_pws_df = zv_included_videos_df.loc[zv_included_videos_df['included_vids'].str.contains('PWS')]
zv_included_pws_count = zv_included_pws_df['included_vids'].count()

# total included videos: fw 
zv_included_fw_df = zv_included_videos_df.loc[zv_included_videos_df['included_vids'].str.contains('FW')]
zv_included_fw_count = zv_included_fw_df['included_vids'].count()

# total_included videos: other tasks? 
zv_included_otherTask_df = zv_included_videos_df.loc[~zv_included_videos_df['included_vids'].str.contains('FW|PWS')]
zv_included_otherTask_count = zv_included_otherTask_df['included_vids'].count()

# total analyzed videos after merge with brainwalk data 
# pws 
zv_analyzed_pws_count = zv_merged_pws_df['id_date_pose_zv'].count()
#fw 
zv_analyzed_fw_count = zv_merged_fw_df['id_date_pose_zv'].count()

# total included (true) - total analyzed (some missing with date issues or wrong task)
zv_included_missing_after_merge = zv_included_vids_count - (zv_analyzed_pws_count + zv_analyzed_fw_count) 

# Home Videos -----------------------------------------------------
# total included videos - from original script, true # of included vids before merging with brainwalk 
hv_included_vids_count = hv_included_videos_df['included_vids'].count()
#right
hv_included_r_df = hv_included_videos_df.loc[zv_included_videos_df['included_vids'].str.contains('gait_vertical_right')]
hv_included_r_count = hv_included_r_df['included_vids'].count()
#left 
hv_included_l_df = hv_included_videos_df.loc[zv_included_videos_df['included_vids'].str.contains('gait_vertical_left')]
hv_included_l_count = hv_included_l_df['included_vids'].count()

# total analyzed after merge w brainwalkd data
hv_analyzed_count = hv_merged_bw_path_df['id_date_pose_hv'].count()
hv_included_missing_after_merge = hv_included_vids_count - hv_analyzed_count

In [16]:
zv_merged_pws_df.columns

Index(['id_date_pose_zv', 'video_id_date_name_pose_zv', 'task_pose_zv',
       'stride_time_mean_sec_pose_zv', 'stride_time_median_sec_pose_zv',
       'stride_time_std_pose_zv', 'stride_time_cv_pose_zv',
       'stride_time_max_pose_zv', 'stride_time_min_pose_zv',
       'mean_cadence_step_per_min_pose_zv', 'stride_width_mean_cm_pose_zv',
       'stride_width_median_cm_pose_zv', 'stride_width_std_pose_zv',
       'stride_width_cv_pose_zv', 'stride_width_max_pose_zv',
       'stride_width_min_pose_zv', 'foot1_gait_cycle_time_mean_pose_zv',
       'foot1_stance_time_mean_pose_zv', 'foot1_stance_per_mean_pose_zv',
       'foot1_swing_time_mean_pose_zv', 'foot1_swing_per_mean_pose_zv',
       'foot1_ini_double_support_time_mean_pose_zv',
       'foot1_term_double_support_time_mean_pose_zv',
       'foot1_tot_double_support_time_mean_pose_zv',
       'foot1_double_support_per_mean_pose_zv',
       'foot1_single_support_time_mean_pose_zv',
       'foot1_single_support_per_mean_pose_zv', 'wa

In [17]:
# Numbers based off of videos analyzed 
    # some true included videos lost when merging brainwalk and need to fix 
    # get 'missing_after_merge' variables to 0

# Zeno---------------------------------
# After running the pose estimation through the final version of the linear walking identification algorithm, 
# at least one linear walking segment was identified in XX PWS and XX FW videos from XX participants with MS 
# and XX PW videos and XX FW videos from  XX healthy controls

# PWS 
    # MS vs HC 
zv_analyzed_pws_MS_df = zv_merged_pws_df.loc[zv_merged_pws_df['demographic_diagnosis'] == 'MS']
zv_analyzed_pws_MS_count = zv_analyzed_pws_MS_df['demographic_diagnosis'].count()

zv_analyzed_pws_HC_df = zv_merged_pws_df.loc[zv_merged_pws_df['demographic_diagnosis'] == 'HC']
zv_analyzed_pws_HC_count = zv_analyzed_pws_HC_df['demographic_diagnosis'].count()

# unique participants with PWS video analyzed
zv_analyzed_pws_MS_participants = zv_analyzed_pws_MS_df['id_video'].nunique()
zv_analyzed_pws_HC_participants = zv_analyzed_pws_HC_df['id_video'].nunique()

# FW 
    # MS vs HC 
zv_analyzed_fw_MS_df = zv_merged_fw_df.loc[zv_merged_fw_df['demographic_diagnosis'] == 'MS']
zv_analyzed_fw_MS_count = zv_analyzed_fw_MS_df['demographic_diagnosis'].count()

zv_analyzed_fw_HC_df = zv_merged_fw_df.loc[zv_merged_fw_df['demographic_diagnosis'] == 'HC']
zv_analyzed_fw_HC_count = zv_analyzed_fw_HC_df['demographic_diagnosis'].count()

# unique participant with FW video analyzed
zv_analyzed_fw_MS_participants = zv_analyzed_fw_MS_df['id_video'].nunique()
zv_analyzed_fw_HC_participants = zv_analyzed_fw_HC_df['id_video'].nunique()

# sum FW and PWS analyzed participants 
# MS 
zv_analyzed_all_MS_df = pd.concat([zv_analyzed_pws_MS_df, zv_analyzed_fw_MS_df])
zv_analyzed_MS_participants = zv_analyzed_all_MS_df['id_video'].nunique()

#HC 
zv_analyzed_all_HC_df = pd.concat([zv_analyzed_pws_HC_df, zv_analyzed_fw_HC_df])
zv_analyzed_HC_participants = zv_analyzed_all_HC_df['id_video'].nunique()

# Home --------------------------------------------------
# After running the pose estimation through the final version of the linear walking identification algorithm, 
# at least one linear walking segment was identified in XX right turning and XX left turning videos from XX participants 

# right vs left 

hv_analysed_r_df = hv_merged_bw_path_df.loc[hv_merged_bw_path_df['task_pose_hv'].str.contains('gait_vertical_right')]
hv_analysed_r_count = hv_analysed_r_df['task_pose_hv'].count()

hv_analysed_l_df = hv_merged_bw_path_df.loc[hv_merged_bw_path_df['task_pose_hv'].str.contains('gait_vertical_left')]
hv_analysed_l_count = hv_analysed_l_df['task_pose_hv'].count()

# unique paricipants with any included videos 
hv_analyzed_all_participants = hv_merged_bw_path_df['id_video'].nunique() 

# participants with right turn 
hv_analyzed_r_participants = hv_analysed_r_df['id_video'].nunique()

# participants with left turn 
hv_analyzed_l_participants = hv_analysed_l_df['id_video'].nunique()

In [18]:
# append values to zeno_video_summary --> .csv file 

zeno_video_summary = []
zeno_video_summary.append({'num_total_zeno_videos' : zv_total_videos,
                           'num_total_pws_zeno_videos' : zv_total_pws_videos,
                           'num_total_fw_zeno_videos' : zv_total_fw_videos, 
                           'num_total_zeno_videos_lostInMerge' : vids_missing_w_bw_merge,
                           'num_total_zeno_vids_w_bw' : zv_bw_total_videos,
                           'num_total_zeno_particpants_wMS' : zv_all_videos_MS_participant_count,
                           'num_total_zeno_pwMS_pws_vids' : zv_all_videos_MS_pws_count, 
                           'num_total_zeno_pwMS_fw_vids' : zv_all_videos_MS_fw_count,
                           'num_total_zeno_particpants_HC' : zv_all_videos_HC_participant_count, 
                           'num_total_zeno_HC_pws_vids' : zv_all_videos_HC_pws_count, 
                           'num_total_zeno_HC_fw_vids' : zv_all_videos_HC_fw_count,
                           'first_total_zeno_vid_date': zv_all_videos_first_date,
                           'last_total_zeno_vid_date': zv_all_videos_last_date, 
                           'total_zeno_vid_visits' : zv_all_unique_visits,
                           'num_total_zeno_ms_multiple_visits' : zv_multiple_visits_ms_count, 
                           'num_total_zeno_ms_single_visits': zv_single_visits_ms_count,
                           'num_total_zeno_hc_multiple_visits' : zv_multiple_visits_hc_count,
                           'num_total_zeno_hc_single_visits' : zv_single_visits_hc_count,
                           'num_included_zeno_vids' : zv_included_vids_count,
                           'num_included_zeno_pws_vids': zv_included_pws_count,
                           'num_included_zeno_fw_vids': zv_included_fw_count,
                           'num_included_zeno_otherTask_vids' : zv_included_otherTask_count,
                           'num_analyzed_zeno_pws_vids' : zv_analyzed_pws_count,
                           'num_analyzed_zeno_fw_vids' : zv_analyzed_fw_count,
                           'num_zv_vids_incl_lostInMerge' : zv_included_missing_after_merge,
                           'num_analyzed_zeno_MS_participants' : zv_analyzed_MS_participants,
                           'num_analyzed_zeno_HC_participants' : zv_analyzed_HC_participants,
                           'num_analyzed_zeno_MS_pws_participants' : zv_analyzed_pws_MS_participants,
                           'num_analyzed_zeno_MS_pws_vids' : zv_analyzed_pws_MS_count,
                           'num_analyzed_zeno_HC_pws_participants' : zv_analyzed_pws_HC_participants, 
                           'num_analyzed_zeno_HC_pws_vids' : zv_analyzed_pws_HC_count,
                           'num_analyzed_zeno_MS_fw_participants' : zv_analyzed_fw_MS_participants, 
                           'num_analyzed_zeno_MS_fw_vids' : zv_analyzed_fw_MS_count,
                           'num_analyzed_zeno_HC_fw_participants' : zv_analyzed_fw_HC_participants,
                           'num_analyzed_zeno_HC_fw_vids' : zv_analyzed_fw_HC_count                       
                          })


zeno_video_summary_df = pd.DataFrame(zeno_video_summary)
zeno_video_summary_df.to_csv(os.path.join(output_path, 'zeno_video_summary_counts.csv'))

In [19]:
# append values to home_video_summary --> .csv file 

home_video_summary = []
home_video_summary.append({'num_total_home_videos' : hv_total_videos,
                           'num_total_right_home_videos' : hv_total_r_videos,
                           'num_total_left_home_videos' : hv_total_l_videos,
                           'num_total_home_particpants' : hv_all_videos_participant_count,
                           'first_total_home_vid_date': hv_all_videos_first_date,
                           'last_total_home_vid_date': hv_all_videos_last_date, 
                           'total_home_vid_visits' : hv_all_unique_visits,
                           'num_total_home_multiple_visits' : hv_multiple_visits_count, 
                           'num_total_home_single_visits': hv_single_visits_count,
                           'num_included_home_vids' : hv_included_vids_count,
                           'num_included_home_right_vids' : hv_included_r_count,
                           'num_included_home_left_vids' : hv_included_l_count,
                           'num_analyzed_home_vids' : hv_analyzed_count,
                           'num_hv_vids_incl_lostInMerge' : hv_included_missing_after_merge, 
                           'num_analyzed_home_participants' : hv_analyzed_all_participants,
                           'num_analyzed_home_right_participants' : hv_analyzed_r_participants, 
                           'num_analyzed_home_right_vids' : hv_analysed_r_count,
                           'num_analyzed_home_left_participants' : hv_analyzed_l_participants,
                           'num_analyzed_home_left_vids' : hv_analysed_l_count}
                         )  

home_video_summary_df = pd.DataFrame(home_video_summary)
home_video_summary_df.to_csv(os.path.join(output_path, 'home_video_summary_counts.csv'))